### Libraries import

In [ ]:
from dotenv import load_dotenv
import os
import requests
import urllib3
import pandas as pd
from tqdm import tqdm
import time
import random
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
pd.set_option('display.max_columns', None)

### Strava API setup

In [ ]:
auth_url = 'https://www.strava.com/oauth/token'
activites_url = 'https://www.strava_com/api/v3/athlete/activities'

load_dotenv()
payload = {
    'client_id': os_getenv('CLIENT_ID'),
    'client_secret': os.getenv('CLIENT_SECRET'),
    'refresh_token': os.getenv('REFRESH_TOKEN'),
    'grant_type': 'refresh_token',
    'f': 'json'
}

### Requesing list of all activities

In [ ]:
print('Requesting Token...\n')
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']

print('Access Token = {}\n'.format(access_token))
header = {'Authorization': 'Bearer ' + access_token}

# The first loop, request_page_number will be set to one, so it requests the first page. Increment this number after
# each request, so the next time we request the second page, then third, and so on...
request_page_num = 1
all_activities = []

while True:
    param = {'per_page': 200, 'page': request_page_num}
    # initial request, where we request the first page of activities
    my_dataset = requests.get(activites_url, headers=header, params=param).json()

    # check the response to make sure it is not empty. If it is empty, that means there is no more data left. So if you have
    # 1000 activities, on the 6th request, where we request page 6, there would be no more data left, so we will break out of the loop
    if len(my_dataset) == 0:
        print('breaking out of while loop because the response is zero, which means there must be no more activities')
        break

    # if the all_activities list is already populated, that means we want to add additional data to it via extend.
    if all_activities:
        print('all_activities is populated')
        all_activities.extend(my_dataset)

    # if the all_activities is empty, this is the first time adding data so we just set it equal to my_dataset
    else:
        print('all_activities is NOT populated')
        all_activities = my_dataset

    request_page_num += 1

print(len(all_activities))
for count, activity in enumerate(all_activities):
    print(activity['name'])
    print(count)

all_activities_df = pd.DataFrame(all_activities)

Requesting Token...

Access Token = 2353fa16ab9db6d2a3a441899381e24f8bfc5791

all_activities is NOT populated
all_activities is populated
all_activities is populated
all_activities is populated
all_activities is populated
all_activities is populated
breaking out of while loop because the response is zero, which means there must be no more activities
1079
Evening Ride
0
Afternoon Weight Training
1
Afternoon Ride
2
Afternoon Walk
3
Afternoon Ride
4
24km Race Practice Long Run🪦
5
Coffee Race 2025☕️
6
9km Easy Run🌅
7
Afternoon Weight Training
8
Afternoon Ride
9
Rolling 300s🪦
10
Afternoon Weight Training
11
Progressive Run🪦
12
22km Long Run🗿
13
Afternoon Weight Training
14
Evening Ride
15
400m Repeats😮‍💨
16
9km Easy Run✨
17
Afternoon Weight Training
18
Afternoon Ride
19
Afternoon Ride
20
11km Long Run🗿
21
Evening Ride
22
Morning Ride
23
Afternoon Walk
24
Afternoon Weight Training
25
Lunch Ride
26
5km Time Trial🕯️
27
Evening Ride🚀
28
5km Easy Run🐝
29
Afternoon Walk
30
Morning Weight Training

### Limiting size of data download

In [178]:
top_n = 25
all_activities_df = all_activities_df.iloc[:top_n]

### Requesing list of all activities with details

In [ ]:
print('Requesting Token...\n')
res = requests.post(auth_url, data=payload, verify=False) # auth_url & payload referenced in code already above
access_token = res.json()['access_token']
header = {'Authorization': 'Bearer ' + access_token}
print('Access Token = {}\n'.format(access_token))

all_activities_details = []
count = 0

print('Calculating time to get all activities details....\n')
print(all_activities_df.shape[0] * 8 / 60, ' minutes to obtain data')

print('Getting details of each activity')
for index, row in tqdm(all_activities_df.iterrows(), total=all_activities_df.shape[0]):
    get_activity_url = 'https://www.strava.com/api/v3/activities/{}'.format(row['id'])
    try:
        activity_details = requests.get(get_activity_url, headers=header).json() 
        all_activities_details.append(activity_details)
        time.sleep(random.randint(7, 9))
    except requests.exception.RequestException as e:
        raise SystemExit(e)

Requesting Token...

Access Token = 2353fa16ab9db6d2a3a441899381e24f8bfc5791

Calculating time to get all activities details....

3.3333333333333335  minutes to obtain data
Getting details of each activity


100%|██████████| 25/25 [03:39<00:00,  8.77s/it]


### All activities Dataframe

In [331]:
all_activities_df.head()

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,map,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_watts,device_watts,kilojoules,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,average_cadence,max_watts,weighted_average_watts
0,2,"{'id': 81055898, 'resource_state': 1}",Evening Ride,14177.5,3095,3461,53.0,Ride,Ride,NaN,15686796063,2025-09-03T17:34:35Z,2025-09-03T19:34:35Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,1,4,0,2,0,"{'id': 'a15686796063', 'summary_polyline': 'ok...",False,False,False,False,followers_only,False,b12572672,"[51.107091, 17.124618]","[51.108163, 17.123457]",4.581,13.54,19.0,100.9,False,312.3,True,103.9,155.0,False,True,127.0,114.4,1.675646e+10,16756458099,garmin_ping_476437881496,False,0,0,False,9.0,NaN,NaN,NaN
1,2,"{'id': 81055898, 'resource_state': 1}",Afternoon Weight Training,0.0,4071,4071,0.0,Workout,WeightTraining,NaN,15683435146,2025-09-03T12:57:35Z,2025-09-03T14:57:35Z,(GMT+02:00) Africa/Blantyre,7200.0,None,None,None,0,3,0,1,0,"{'id': 'a15683435146', 'summary_polyline': '',...",True,False,False,False,followers_only,False,None,[],[],0.000,0.00,NaN,NaN,NaN,NaN,True,97.9,137.0,False,True,0.0,0.0,1.675291e+10,16752910671,garmin_ping_476360234705,False,0,0,False,9.0,NaN,NaN,NaN
2,2,"{'id': 81055898, 'resource_state': 1}",Afternoon Ride,11833.2,1778,7158,32.0,Ride,Ride,10.0,15683575988,2025-09-03T12:37:26Z,2025-09-03T14:37:26Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,1,4,0,1,0,"{'id': 'a15683575988', 'summary_polyline': '{t...",False,True,False,False,followers_only,False,b12572672,"[51.107588, 17.123967]","[51.108269, 17.123584]",6.655,10.04,26.0,158.1,False,281.0,True,134.1,149.0,False,True,130.4,117.2,1.675307e+10,16753067661,garmin_ping_476364154361,False,0,0,False,13.0,NaN,NaN,NaN
3,2,"{'id': 81055898, 'resource_state': 1}",Afternoon Walk,5062.4,3664,3840,12.0,Walk,Walk,NaN,15660079944,2025-09-01T14:19:56Z,2025-09-01T16:19:56Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,0,8,0,2,0,"{'id': 'a15660079944', 'summary_polyline': 'wh...",False,False,False,False,followers_only,False,None,"[51.107714, 17.124284]","[51.108194, 17.125157]",1.382,1.88,NaN,NaN,NaN,NaN,True,81.2,108.0,False,True,124.2,116.2,1.672690e+10,16726897935,garmin_ping_475670331039,False,0,0,False,5.0,52.4,NaN,NaN
4,2,"{'id': 81055898, 'resource_state': 1}",Afternoon Ride,24309.7,5770,7708,63.0,Ride,Ride,10.0,15650786884,2025-08-31T13:43:42Z,2025-08-31T15:43:42Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,11,13,0,2,0,"{'id': 'a15650786884', 'summary_polyline': 'ql...",False,False,False,False,followers_only,False,b12572672,"[51.10669, 17.123393]","[51.107619, 17.124172]",4.213,6.66,23.0,82.4,False,475.7,True,93.8,119.0,False,True,125.2,113.2,1.671661e+10,16716611559,garmin_ping_475336836414,False,4,0,False,12.0,NaN,NaN,NaN


### All activities with details Dataframe

In [330]:
all_activities_details_df = pd.json_normalize(all_activities_details, sep='.')
all_activities_details_df.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_watts,device_watts,kilojoules,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,description,calories,perceived_exertion,prefer_perceived_exertion,segment_efforts,splits_metric,splits_standard,laps,stats_visibility,hide_from_home,device_name,embed_token,available_zones,athlete.id,athlete.resource_state,map.id,map.polyline,map.resource_state,map.summary_polyline,gear.id,gear.primary,gear.name,gear.nickname,gear.resource_state,gear.retired,gear.distance,gear.converted_distance,photos.primary,photos.count,average_cadence,max_watts,weighted_average_watts,best_efforts,similar_activities.effort_count,similar_activities.average_speed,similar_activities.min_average_speed,similar_activities.mid_average_speed,similar_activities.max_average_speed,similar_activities.pr_rank,similar_activities.frequency_milestone,similar_activities.trend.speeds,similar_activities.trend.current_activity_index,similar_activities.trend.min_speed,similar_activities.trend.mid_speed,similar_activities.trend.max_speed,similar_activities.trend.direction,similar_activities.resource_state,photos.primary.unique_id,photos.primary.urls.600,photos.primary.urls.100,photos.primary.source,photos.primary.media_type,photos.use_primary_photo
0,3,Evening Ride,14177.5,3095,3461,53.0,Ride,Ride,NaN,15686796063,2025-09-03T17:34:35Z,2025-09-03T19:34:35Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,1,4,0,2,0,False,False,False,False,followers_only,False,b12572672,"[51.107091, 17.124618]","[51.108163, 17.123457]",4.581,13.54,19.0,100.9,False,312.3,True,103.9,155.0,False,True,127.0,114.4,16756458099,16756458099,garmin_ping_476437881496,False,0,0,False,9.0,Szybki test nowego setupu🗿,274.0,None,None,"[{'id': 3398400794639925554, 'resource_state':...","[{'distance': 1000.5, 'elapsed_time': 315, 'el...","[{'distance': 1613.0, 'elapsed_time': 454, 'el...","[{'id': 55856340694, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin Edge 840,dcab37d8f594ea69e901d6eb3b68c52d552a744d,"[heartrate, power]",81055898,1,a15686796063,iz|vHysogB^g@FEF?f@hAHDX@BBF^`@x@Pn@T\D@jAiAh@...,3,ok}vHw_ngBUZ[n@cAbBaAzAa@r@a@j@q@pAe@p@}@`BIHs...,b12572672,False,Cube Nuroad Pro,Cube Nuroad Pro,2.0,False,3625151.0,3625.2,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Afternoon Weight Training,0.0,4071,4071,0.0,Workout,WeightTraining,NaN,15683435146,2025-09-03T12:57:35Z,2025-09-03T14:57:35Z,(GMT+02:00) Africa/Blantyre,7200.0,None,None,None,0,3,0,1,0,True,False,False,False,followers_only,False,None,[],[],0.000,0.00,NaN,NaN,NaN,NaN,True,97.9,137.0,False,True,0.0,0.0,16752910671,16752910671,garmin_ping_476360234705,False,0,0,False,9.0,Reska8️⃣4️⃣,330.0,None,None,[],NaN,NaN,"[{'id': 55843284355, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin Forerunner 970,af5d26a5609de65ebdc6f37c9c01da44505d3187,[heartrate],81055898,1,a15683435146,,3,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Afternoon Ride,11833.2,1778,7158,32.0,Ride,Ride,10.0,15683575988,2025-09-03T12:37:26Z,2025-09-03T14:37:26Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,1,4,0,1,0,False,True,False,False,followers_only,False,b12572672,"[51.107588, 17.123967]","[51.108269, 17.123584]",6.655,10.04,26.0,158.1,False,281.0,True,134.1,149.0,False,True,130.4,117.2,16753067

### Separate tables setup

In [310]:
dataframe_columns = {
  'activities' : [
    'id',
    'name',
    'distance',
    'moving_time',
    'elapsed_time',
    'total_elevation_gain',
    'type',
    'sport_type',
    'workout_type',
    'start_date',
    'start_date_local',
    'timezone',
    'utc_offset',
    'location_city',
    'location_state',
    'location_country',
    'achievement_count',
    'kudos_count',
    'comment_count',
    'athlete_count',
    'photo_count',
    'trainer',
    'commute',
    'manual',
    'private',
    'visibility',
    'flagged',
    'gear_id',
    'start_latlng',
    'end_latlng',
    'average_speed',
    'max_speed',
    'average_cadence',
    'average_watts',
    'max_watts',
    'weighted_average_watts',
    'device_watts',
    'kilojoules',
    'has_heartrate',
    'average_heartrate',
    'max_heartrate',
    'heartrate_opt_out',
    'display_hide_heartrate_option',
    'elev_high',
    'elev_low',
    'upload_id',
    'upload_id_str',
    'external_id',
    'from_accepted_tag',
    'pr_count',
    'total_photo_count',
    'has_kudoed',
    'suffer_score',
    'description',
    'calories',
    'perceived_exertion',
    'prefer_perceived_exertion',
    'hide_from_home',
    'device_name',
    'embed_token',
    'available_zones',
    'map_id',
    'gear_id'],
  'maps' : [
    'map_id',
    'map_polyline',
    'map_resource_state',
    'map_summary_polyline'],
  'gear' : [
    'gear_id',
    'gear_primary',
    'gear_name',
    'gear_nickname',
    'gear_resource_state',
    'gear_retired',
    'gear_distance',
    'gear_converted_distance'],
  'segment_efforts' : [
    'id',
    'resource_state',
    'name',
    'elapsed_time',
    'moving_time',
    'start_date',
    'start_date_local',
    'distance',
    'start_index',
    'end_index',
    'average_cadence',
    'device_watts',
    'average_watts',
    'average_heartrate',
    'max_heartrate',
    'pr_rank',
    'achievements',
    'visibility',
    'kom_rank',
    'hidden',
    'activity_id',
    'segment_id'],
  'segments' : [
    'segment_id',
    'segment_resource_state',
    'segment_name',
    'segment_activity_type',
    'segment_distance',
    'segment_average_grade',
    'segment_maximum_grade',
    'segment_elevation_high',
    'segment_elevation_low',
    'segment_start_latlng',
    'segment_end_latlng',
    'segment_elevation_profile',
    'segment_elevation_profiles',
    'segment_climb_category',
    'segment_city',
    'segment_state',
    'segment_country',
    'segment_private',
    'segment_hazardous',
    'segment_starred'],
  'laps' : [
    'id',
    'resource_state',
    'name',
    'elapsed_time',
    'moving_time',
    'start_date',
    'start_date_local',
    'distance',
    'average_speed',
    'max_speed',
    'lap_index',
    'split',
    'start_index',
    'end_index',
    'total_elevation_gain',
    'average_cadence',
    'device_watts',
    'average_watts',
    'average_heartrate',
    'max_heartrate',
    'pace_zone',
    'activity_id'],
  'best_efforts' : [
    'id',
    'activity_id',
    'resource_state',
    'name',
    'elapsed_time',
    'moving_time',
    'start_date',
    'start_date_local',
    'distance',
    'pr_rank',
    'achievements',
    'start_index',
    'end_index']
}

### Spliting data into tables

In [332]:
# Top level normalize
all_activities_details_normalized_df = pd.json_normalize(all_activities_details, sep='_')

# Activities
activities_cols = dataframe_columns['activities']
activities_df = all_activities_details_normalized_df[[c for c in activities_cols if c in all_activities_details_normalized_df.columns]].copy()

# Maps
maps_cols = dataframe_columns['maps']
maps_df = all_activities_details_normalized_df[[c for c in maps_cols if c in all_activities_details_normalized_df.columns]].copy()

# Gear
gear_cols = dataframe_columns['gear']
gear_df = all_activities_details_normalized_df[[c for c in gear_cols if c in all_activities_details_normalized_df.columns]].copy()

# Segment efforts
seg_eff_cols = dataframe_columns['segment_efforts']
segments_eff_df = pd.json_normalize(
    all_activities_details,
    record_path='segment_efforts',
    sep='_'
)
if not segments_eff_df.empty:
    segments_eff_df = segments_eff_df[[c for c in seg_eff_cols if c in segments_eff_df.columns]].copy()

# Segments
seg_cols = dataframe_columns['segments']
segments_df = pd.json_normalize(
    all_activities_details,
    record_path='segment_efforts',
    sep="_"
)

if not segments_df.empty:
    segments_df = segments_df[[c for c in seg_cols if c in segments_df.columns]].copy()

# Laps
lap_cols = dataframe_columns['laps']
laps_df = pd.json_normalize(
    all_activities_details,
    record_path='laps',
    sep="_"
)
if not laps_df.empty:
    laps_df = laps_df[[c for c in lap_cols if c in laps_df.columns]]

# Best efforts
best_eff_cols = dataframe_columns['best_efforts']
activities_with_best_eff = [row for row in all_activities_details if row.get('best_efforts')]
best_efforts_df = pd.json_normalize(
    activities_with_best_eff,
    record_path='best_efforts',
    sep="_",
)

if not best_efforts_df.empty:
    best_efforts_df = best_efforts_df[[c for c in best_eff_cols if c in best_efforts_df.columns]]

# All dataframes in dictoinary
dataframes = {
    "activities": activities_df,
    "maps": maps_df,
    "gear": gear_df,
    "segment_efforts": segments_eff_df,
    "segments": segments_df,
    "laps": laps_df,
    "best_efforts": best_efforts_df
}

### Data cleanup